# Neighborhoods in Toronto - part 3
- Explore and cluster the neighborhoods in Toronto.

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors

import folium

from sklearn.cluster import KMeans

### Load the geographical coordinates of neighborhoods from part 2 

In [2]:
toronto_coords = pd.read_csv('Toronto_coordinates.csv')
toronto_coords.head()

PostalCode      Borough                            Neighborhood   Latitude  \
0        M1B  Scarborough                          Malvern, Rouge  43.806686   
1        M1C  Scarborough  Rouge Hill, Port Union, Highland Creek  43.784535   
2        M1E  Scarborough       Guildwood, Morningside, West Hill  43.763573   
3        M1G  Scarborough                                  Woburn  43.770992   
4        M1H  Scarborough                               Cedarbrae  43.773136   

   Longitude  
0 -79.194353  
1 -79.160497  
2 -79.188711  
3 -79.216917  
4 -79.239476

#### Use geopy library to get the latitude and longitude values of Toronto City

In [3]:
from geopy.geocoders import Nominatim

address = 'Toronto Canada'
geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

print(f'The geograpical coordinate of Toronto City are {latitude}, {longitude}.')

The geograpical coordinate of Toronto City are 43.6534817, -79.3839347.


#### Create a map of Toronto with neighborhoods superimposed on top.

In [4]:
import folium

# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_coords['Latitude'], toronto_coords['Longitude'],
                                           toronto_coords['Borough'], toronto_coords['Neighborhood']):
    label = f'{neighborhood}, {borough}'
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat, lng], 
                        radius=5,
                        popup=label,
                        color='blue',
                        fill=True,
                        fill_color='#3186cc',
                        fill_opacity=0.7,
                        parse_html=False).add_to(map_toronto)


# Display the map
map_toronto

#### Work with only boroughs that contain the word "Toronto"

In [5]:
toronto_coords['Borough'][toronto_coords['Borough'].str.contains('Toronto')].value_counts()

Downtown Toronto    19
Central Toronto      9
West Toronto         6
East Toronto         5
Name: Borough, dtype: int64

In [6]:
borough_toronto = toronto_coords[toronto_coords['Borough'].str.contains('Toronto')].sort_values(by=['PostalCode']).reset_index(drop=True)
print(borough_toronto.shape)
borough_toronto.head()

(39, 5)


PostalCode          Borough                    Neighborhood   Latitude  \
0        M4E     East Toronto                     The Beaches  43.676357   
1        M4K     East Toronto    The Danforth West, Riverdale  43.679557   
2        M4L     East Toronto  India Bazaar, The Beaches West  43.668999   
3        M4M     East Toronto                 Studio District  43.659526   
4        M4N  Central Toronto                   Lawrence Park  43.728020   

   Longitude  
0 -79.293031  
1 -79.352188  
2 -79.315572  
3 -79.340923  
4 -79.388790

#### Create a map of boroughs that contain the word "Toronto" 

In [7]:
import folium

# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(borough_toronto['Latitude'], borough_toronto['Longitude'],
                                           borough_toronto['Borough'], borough_toronto['Neighborhood']):
    label = f'{neighborhood}, {borough}'
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat, lng], 
                        radius=5,
                        popup=label,
                        color='blue',
                        fill=True,
                        fill_color='#3186cc',
                        fill_opacity=0.7,
                        parse_html=False).add_to(map_toronto)


# Display the map
map_toronto

### Use the Foursquare API to explore the neighborhoods

In [8]:
CLIENT_ID = 'C2XJXBVHYV1HFNHGDBEXEQPWLXRJA3KHJZGX53EQEPHTO5SZ' # your Foursquare ID
CLIENT_SECRET = '02LZ0P3GVPQXG4MTY53Q1NQDF4QU5AWXMRCTBSNHFPL2RFBQ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: C2XJXBVHYV1HFNHGDBEXEQPWLXRJA3KHJZGX53EQEPHTO5SZ
CLIENT_SECRET:02LZ0P3GVPQXG4MTY53Q1NQDF4QU5AWXMRCTBSNHFPL2RFBQ


#### Let's explore the first neighborhood in our dataframe.

Get the neighborhood's name.

In [9]:
borough_toronto.loc[0, 'Neighborhood']

'The Beaches'

Get the neighborhood's latitude and longitude values.

In [11]:
neighborhood_latitude = borough_toronto.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = borough_toronto.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = borough_toronto.loc[0, 'Neighborhood'] # neighborhood name

print(f'Latitude and longitude values of {neighborhood_name} are {neighborhood_latitude}, {neighborhood_longitude}.')

Latitude and longitude values of The Beaches are 43.67635739999999, -79.2930312.


#### Let's get the top 100 venues that are within a radius of 500 meters from the first neighborhood.

In [12]:
LIMIT = 100
radius = 500

# create url
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=C2XJXBVHYV1HFNHGDBEXEQPWLXRJA3KHJZGX53EQEPHTO5SZ&client_secret=02LZ0P3GVPQXG4MTY53Q1NQDF4QU5AWXMRCTBSNHFPL2RFBQ&v=20180605&ll=43.67635739999999,-79.2930312&radius=500&limit=100'

Send the GET request and examine the resutls

In [13]:
import requests # library to handle requests

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f015797889e0936198e0588'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'The Beaches',
  'headerFullLocation': 'The Beaches, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 5,
  'suggestedBounds': {'ne': {'lat': 43.680857404499996,
    'lng': -79.28682091449052},
   'sw': {'lat': 43.67185739549999, 'lng': -79.29924148550948}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bd461bc77b29c74a07d9282',
       'name': 'Glen Manor Ravine',
       'location': {'address': 'Glen Manor',
        'crossStreet': 'Queen St.',
        'lat': 43.67682094413784,
        'lng': -79.29394208780985,
        'labeledLatLngs': [{'labe

All the information is in the items key. Before we proceed, let's borrow the **get_category_type** function from the Foursquare lab.

In [14]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

clean the json and structure it into a pandas dataframe.

In [15]:
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

venues = results['response']['groups'][0]['items']

nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

name         categories        lat        lng
0                   Glen Manor Ravine              Trail  43.676821 -79.293942
1  The Big Carrot Natural Food Market  Health Food Store  43.678879 -79.297734
2                 Grover Pub and Grub                Pub  43.679181 -79.297215
3                       Upper Beaches       Neighborhood  43.680563 -79.292869
4                 Seaspray Restaurant   Asian Restaurant  43.678888 -79.298167

And how many venues were returned by Foursquare?

In [16]:
print(f'{nearby_venues.shape[0]} venues were returned by Foursquare.')

5 venues were returned by Foursquare.


### Explore Neighborhoods in Toronto

#### Let's create a function to repeat the same process to all the neighborhoods in Toronto

In [17]:
# get the top 100 venues that are within a radius of 500 meters.
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Run the above function on each neighborhood and create a new dataframe called *toronto_venues*.

In [18]:
toronto_venues = getNearbyVenues(names=borough_toronto['Neighborhood'],
                                   latitudes=borough_toronto['Latitude'],
                                   longitudes=borough_toronto['Longitude']
                                  )


The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West, Lawrence Park
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West, Forest Hill Road Park
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High 

#### Let's check the size of the resulting dataframe

In [19]:
print(toronto_venues.shape)
toronto_venues.head()

(1614, 7)


Neighborhood  Neighborhood Latitude  Neighborhood Longitude  \
0  The Beaches              43.676357              -79.293031   
1  The Beaches              43.676357              -79.293031   
2  The Beaches              43.676357              -79.293031   
3  The Beaches              43.676357              -79.293031   
4  The Beaches              43.676357              -79.293031   

                                Venue  Venue Latitude  Venue Longitude  \
0                   Glen Manor Ravine       43.676821       -79.293942   
1  The Big Carrot Natural Food Market       43.678879       -79.297734   
2                 Grover Pub and Grub       43.679181       -79.297215   
3                       Upper Beaches       43.680563       -79.292869   
4                 Seaspray Restaurant       43.678888       -79.298167   

      Venue Category  
0              Trail  
1  Health Food Store  
2                Pub  
3       Neighborhood  
4   Asian Restaurant

In [20]:
toronto_venues.to_csv('Toronto_venues.csv', index=False)

Let's check how many venues were returned for each neighborhood

In [21]:
toronto_venues.groupby('Neighborhood').count()

Neighborhood Latitude  \
Neighborhood                                                                
Berczy Park                                                            58   
Brockton, Parkdale Village, Exhibition Place                           22   
Business reply mail Processing Centre, South Ce...                     19   
CN Tower, King and Spadina, Railway Lands, Harb...                     15   
Central Bay Street                                                     65   
Christie                                                               17   
Church and Wellesley                                                   75   
Commerce Court, Victoria Hotel                                        100   
Davisville                                                             35   
Davisville North                                                        8   
Dufferin, Dovercourt Village                                           15   
First Canadian Place, Underground city                                100   
Forest Hill North & West, Forest Hill Road Park                         4   
Garden District, Ryerson                                              100   
Harbourfront East, Union Station, Toronto Islands                     100   
High Park, The Junction South                                          25   
India Bazaar, The Beaches West                                         23   
Kensington Market, Chinatown, Grange Park                              60   
Lawrence Park                                                           3   
Little Portugal, Trinity                                               44   
Moore Park, Summerhill East                                             1   
North Toronto West, Lawrence Park                                      18   
Parkdale, Roncesvalles                                                 15   
Queen's Park, Ontario Provincial Government                            32   
Regent Park, Harbourfront                                              44   
Richmond, Adelaide, King                                               94   
Rosedale                                                                4   
Roselawn                                                                3   
Runnymede, Swansea                                                     35   
St. James Town                                                         79   
St. James Town, Cabbagetown                                            42   
Stn A PO Boxes                                                         97   
Studio District                                                        42   
Summerhill West, Rathnelly, South Hill, Forest ...                     16   
The Annex, North Midtown, Yorkville                                    21   
The Beaches                                                             5   
The Danforth West, Riverdale                                           43   
Toronto Dominion Centre, Design Exchange                              100   
University of Toronto, Harbord                                         35   

                                                    Neighborhood Longitude  \
Neighborhood                                                                 
Berczy Park                                                             58   
Brockton, Parkdale Village, Exhibition Place                            22   
Business reply mail Processing Centre, South Ce...                      19   
CN Tower, King and Spadina, Railway Lands, Harb...                      15   
Central Bay Street                                                      65   
Christie                                                                17   
Church and Wellesley                                                    75   
Commerce Court, Victoria Hotel                                         100   
Davisville                                                              35   
Davisville North                    

In [22]:
toronto_venues['Neighborhood'].value_counts()

Toronto Dominion Centre, Design Exchange                                                                      100
Garden District, Ryerson                                                                                      100
Harbourfront East, Union Station, Toronto Islands                                                             100
Commerce Court, Victoria Hotel                                                                                100
First Canadian Place, Underground city                                                                        100
Stn A PO Boxes                                                                                                 97
Richmond, Adelaide, King                                                                                       94
St. James Town                                                                                                 79
Church and Wellesley                                                                    

#### Let's find out how many unique categories can be curated from all the returned venues

In [23]:
toronto_venues['Venue Category'].value_counts()

Coffee Shop                        143
Café                                89
Restaurant                          54
Italian Restaurant                  41
Hotel                               37
Park                                34
Japanese Restaurant                 33
Bakery                              33
Pizza Place                         30
Bar                                 27
Gym                                 25
Sushi Restaurant                    25
Sandwich Place                      24
Pub                                 22
American Restaurant                 21
Seafood Restaurant                  21
Clothing Store                      20
Thai Restaurant                     19
Beer Bar                            18
Vegetarian / Vegan Restaurant       17
Diner                               17
Bookstore                           17
Gastropub                           17
Grocery Store                       16
Breakfast Spot                      16
Ice Cream Shop           

In [24]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 233 uniques categories.


###  Analyze Each Neighborhood

In [25]:
toronto_venues.head()

Neighborhood  Neighborhood Latitude  Neighborhood Longitude  \
0  The Beaches              43.676357              -79.293031   
1  The Beaches              43.676357              -79.293031   
2  The Beaches              43.676357              -79.293031   
3  The Beaches              43.676357              -79.293031   
4  The Beaches              43.676357              -79.293031   

                                Venue  Venue Latitude  Venue Longitude  \
0                   Glen Manor Ravine       43.676821       -79.293942   
1  The Big Carrot Natural Food Market       43.678879       -79.297734   
2                 Grover Pub and Grub       43.679181       -79.297215   
3                       Upper Beaches       43.680563       -79.292869   
4                 Seaspray Restaurant       43.678888       -79.298167   

      Venue Category  
0              Trail  
1  Health Food Store  
2                Pub  
3       Neighborhood  
4   Asian Restaurant

####  One hot encoding for 'Venue Category' column and add 'Neighborhood' column in new dataframe

In [26]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhoods'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]


toronto_onehot.head()

Neighborhoods  Afghan Restaurant  Airport  Airport Food Court  \
0   The Beaches                  0        0                   0   
1   The Beaches                  0        0                   0   
2   The Beaches                  0        0                   0   
3   The Beaches                  0        0                   0   
4   The Beaches                  0        0                   0   

   Airport Lounge  Airport Service  Airport Terminal  American Restaurant  \
0               0                0                 0                    0   
1               0                0                 0                    0   
2               0                0                 0                    0   
3               0                0                 0                    0   
4               0                0                 0                    0   

   Antique Shop  Aquarium  Art Gallery  Art Museum  Arts & Crafts Store  \
0             0         0            0           0                    0   
1             0         0            0           0                    0   
2             0         0            0           0                    0   
3             0         0            0           0                    0   
4             0         0            0           0                    0   

   Asian Restaurant  Athletics & Sports  Auto Workshop  BBQ Joint  Baby Store  \
0                 0                   0              0          0           0   
1                 0                   0              0          0           0   
2                 0                   0              0          0           0   
3                 0                   0              0          0           0   
4                 1                   0              0          0           0   

   Bagel Shop  Bakery  Bank  Bar  Baseball Stadium  Basketball Stadium  Beach  \
0           0       0     0    0                 0                   0      0   
1           0       0     0    0                 0                   0      0   
2           0       0     0    0                 0                   0      0   
3           0       0     0    0                 0                   0      0   
4           0       0     0    0                 0                   0      0   

   Bed & Breakfast  Beer Bar  Beer Store  Belgian Restaurant  Bistro  \
0                0         0           0                   0       0   
1                0         0           0                   0       0   
2                0         0           0                   0       0   
3                0         0           0                   0       0   
4                0         0           0                   0       0   

   Boat or Ferry  Bookstore  Boutique  Brazilian Restaurant  Breakfast Spot  \
0              0          0         0                     0               0   
1              0          0         0                     0               0   
2              0          0         0                     0               0   
3              0          0         0                     0               0   
4              0          0         0                     0               0   

   Brewery  Bubble Tea Shop  Building  Burger Joint  Burrito Place  Bus Line  \
0        0                0         0             0              0         0   
1        0                0         0             0              0         0   
2        0                0         0             0              0         0   
3        0                0         0             0              0         0   
4        0                0         0             0              0         0   

   Butcher  Café  Cajun / Creole Restaurant  Camera Store  Candy Store  \
0        0     0                          0             0            0   
1        0     0                          0             0            0   
2        0     0                          0             0            0   
3        0     0          

In [27]:
toronto_onehot.shape

(1614, 234)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [28]:
toronto_grouped = toronto_onehot.groupby('Neighborhoods').mean().reset_index()

toronto_grouped

Neighborhoods  Afghan Restaurant  \
0                                         Berczy Park           0.000000   
1        Brockton, Parkdale Village, Exhibition Place           0.000000   
2   Business reply mail Processing Centre, South C...           0.000000   
3   CN Tower, King and Spadina, Railway Lands, Har...           0.000000   
4                                  Central Bay Street           0.000000   
5                                            Christie           0.000000   
6                                Church and Wellesley           0.013333   
7                      Commerce Court, Victoria Hotel           0.000000   
8                                          Davisville           0.000000   
9                                    Davisville North           0.000000   
10                       Dufferin, Dovercourt Village           0.000000   
11             First Canadian Place, Underground city           0.000000   
12    Forest Hill North & West, Forest Hill Road Park           0.000000   
13                           Garden District, Ryerson           0.000000   
14  Harbourfront East, Union Station, Toronto Islands           0.000000   
15                      High Park, The Junction South           0.000000   
16                     India Bazaar, The Beaches West           0.000000   
17          Kensington Market, Chinatown, Grange Park           0.000000   
18                                      Lawrence Park           0.000000   
19                           Little Portugal, Trinity           0.000000   
20                        Moore Park, Summerhill East           0.000000   
21                  North Toronto West, Lawrence Park           0.000000   
22                             Parkdale, Roncesvalles           0.000000   
23        Queen's Park, Ontario Provincial Government           0.000000   
24                          Regent Park, Harbourfront           0.000000   
25                           Richmond, Adelaide, King           0.000000   
26                                           Rosedale           0.000000   
27                                           Roselawn           0.000000   
28                                 Runnymede, Swansea           0.000000   
29                                     St. James Town           0.000000   
30                        St. James Town, Cabbagetown           0.000000   
31                                     Stn A PO Boxes           0.000000   
32                                    Studio District           0.000000   
33  Summerhill West, Rathnelly, South Hill, Forest...           0.000000   
34                The Annex, North Midtown, Yorkville           0.000000   
35                                        The Beaches           0.000000   
36                       The Danforth West, Riverdale           0.000000   
37           Toronto Dominion Centre, Design Exchange           0.000000   
38                     University of Toronto, Harbord           0.000000   

     Airport  Airport Food Court  Airport Lounge  Airport Service  \
0   0.000000            0.000000        0.000000              0.0   
1   0.000000            0.000000        0.000000              0.0   
2   0.000000            0.000000        0.000000              0.0   
3   0.066667            0.066667        0.133333              0.2   
4   0.000000            0.000000        0.000000              0.0   
5   0.000000            0.000000        0.000000              0.0   
6   0.000000            0.000000        0.000000              0.0   
7   0.000000            0.000000        0.000000              0.0   
8   0.000000            0.000000        0.000000              0.0   
9   0.000000            0.000000        0.000000              0.0   
10  0.000000            0.000000        0.000000              0.0   
11  0.000000            0.000000        0.000000              0.0   
12  0.000000            0.000000        0.000000              0.0   
13  0.000000            0.000000 

In [29]:
toronto_grouped.shape

(39, 234)

#### Let's print each neighborhood along with the top 5 most common venues

In [30]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhoods']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhoods'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.09
1        Cocktail Bar  0.05
2                Café  0.03
3  Seafood Restaurant  0.03
4         Cheese Shop  0.03


----Brockton, Parkdale Village, Exhibition Place----
            venue  freq
0            Café  0.14
1  Breakfast Spot  0.09
2     Coffee Shop  0.09
3    Intersection  0.05
4         Stadium  0.05


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                venue  freq
0  Light Rail Station  0.11
1          Comic Shop  0.05
2       Garden Center  0.05
3         Pizza Place  0.05
4    Recording Studio  0.05


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                venue  freq
0     Airport Service  0.20
1      Airport Lounge  0.13
2     Harbor / Marina  0.07
3  Airport Food Court  0.07
4    Airport Terminal  0.07


----Central Bay Street----
                 venue  freq
0      

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [31]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [32]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhoods']

for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))


# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhoods'] = toronto_grouped['Neighborhoods']


for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

Neighborhoods 1st Most Common Venue  \
0                                        Berczy Park           Coffee Shop   
1       Brockton, Parkdale Village, Exhibition Place                  Café   
2  Business reply mail Processing Centre, South C...    Light Rail Station   
3  CN Tower, King and Spadina, Railway Lands, Har...       Airport Service   
4                                 Central Bay Street           Coffee Shop   

  2nd Most Common Venue 3rd Most Common Venue  4th Most Common Venue  \
0          Cocktail Bar    Seafood Restaurant                 Bakery   
1           Coffee Shop        Breakfast Spot  Performing Arts Venue   
2            Smoke Shop           Pizza Place             Restaurant   
3        Airport Lounge       Harbor / Marina                    Bar   
4        Sandwich Place    Italian Restaurant    Japanese Restaurant   

  5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
0            Restaurant                  Café           Cheese Shop   
1                Bakery             Nightclub             Pet Store   
2               Butcher               Brewery  Fast Food Restaurant   
3           Coffee Shop   Rental Car Location      Sculpture Garden   
4                  Café       Thai Restaurant      Department Store   

  8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
0              Beer Bar      Department Store              Nightclub  
1          Climbing Gym            Restaurant          Burrito Place  
2         Auto Workshop                   Spa         Farmers Market  
3         Boat or Ferry              Boutique       Airport Terminal  
4                   Bar           Salad Place        Bubble Tea Shop

In [33]:
neighborhoods_venues_sorted

Neighborhoods 1st Most Common Venue  \
0                                         Berczy Park           Coffee Shop   
1        Brockton, Parkdale Village, Exhibition Place                  Café   
2   Business reply mail Processing Centre, South C...    Light Rail Station   
3   CN Tower, King and Spadina, Railway Lands, Har...       Airport Service   
4                                  Central Bay Street           Coffee Shop   
5                                            Christie         Grocery Store   
6                                Church and Wellesley           Coffee Shop   
7                      Commerce Court, Victoria Hotel           Coffee Shop   
8                                          Davisville           Pizza Place   
9                                    Davisville North                   Gym   
10                       Dufferin, Dovercourt Village              Pharmacy   
11             First Canadian Place, Underground city           Coffee Shop   
12    Forest Hill North & West, Forest Hill Road Park         Jewelry Store   
13                           Garden District, Ryerson        Clothing Store   
14  Harbourfront East, Union Station, Toronto Islands           Coffee Shop   
15                      High Park, The Junction South         Grocery Store   
16                     India Bazaar, The Beaches West                  Park   
17          Kensington Market, Chinatown, Grange Park                  Café   
18                                      Lawrence Park                  Park   
19                           Little Portugal, Trinity                   Bar   
20                        Moore Park, Summerhill East                  Park   
21                  North Toronto West, Lawrence Park           Coffee Shop   
22                             Parkdale, Roncesvalles        Breakfast Spot   
23        Queen's Park, Ontario Provincial Government           Coffee Shop   
24                          Regent Park, Harbourfront           Coffee Shop   
25                           Richmond, Adelaide, King           Coffee Shop   
26                                           Rosedale                  Park   
27                                           Roselawn                  Pool   
28                                 Runnymede, Swansea                  Café   
29                                     St. James Town           Coffee Shop   
30                        St. James Town, Cabbagetown                Bakery   
31                                     Stn A PO Boxes           Coffee Shop   
32                                    Studio District                  Café   
33  Summerhill West, Rathnelly, South Hill, Forest...                   Pub   
34                The Annex, North Midtown, Yorkville                  Café   
35                                        The Beaches     Health Food Store   
36                       The Danforth West, Riverdale      Greek Restaurant   
37           Toronto Dominion Centre, Design Exchange           Coffee Shop   
38                     University of Toronto, Harbord                  Café   

   2nd Most Common Venue  3rd Most Common Venue  \
0           Cocktail Bar     Seafood Restaurant   
1            Coffee Shop         Breakfast Spot   
2             Smoke Shop            Pizza Place   
3         Airport Lounge        Harbor / Marina   
4         Sandwich Place     Italian Restaurant   
5                   Café                   Park   
6       Sushi Restaurant    Japanese Restaurant   
7                   Café             Restaurant   
8           Dessert Shop         Sandwich Place   
9                  Hotel       Department Store   
10                Bakery          Grocery Store   
11                  Café             Restaurant   
12                 Trail     Mexican Restaurant   
13           Coffee Shop                   Café   
14              Aquarium                  Hotel   
15                  Café     Mexican Restaurant   
16        Sa

In [34]:
neighborhoods_venues_sorted.shape

(39, 11)

## Cluster Neighborhoods

In [35]:
toronto_grouped.head()

Neighborhoods  Afghan Restaurant  \
0                                        Berczy Park                0.0   
1       Brockton, Parkdale Village, Exhibition Place                0.0   
2  Business reply mail Processing Centre, South C...                0.0   
3  CN Tower, King and Spadina, Railway Lands, Har...                0.0   
4                                 Central Bay Street                0.0   

    Airport  Airport Food Court  Airport Lounge  Airport Service  \
0  0.000000            0.000000        0.000000              0.0   
1  0.000000            0.000000        0.000000              0.0   
2  0.000000            0.000000        0.000000              0.0   
3  0.066667            0.066667        0.133333              0.2   
4  0.000000            0.000000        0.000000              0.0   

   Airport Terminal  American Restaurant  Antique Shop  Aquarium  Art Gallery  \
0          0.000000                  0.0           0.0       0.0     0.017241   
1          0.000000                  0.0           0.0       0.0     0.000000   
2          0.000000                  0.0           0.0       0.0     0.000000   
3          0.066667                  0.0           0.0       0.0     0.000000   
4          0.000000                  0.0           0.0       0.0     0.000000   

   Art Museum  Arts & Crafts Store  Asian Restaurant  Athletics & Sports  \
0    0.000000                  0.0               0.0                 0.0   
1    0.000000                  0.0               0.0                 0.0   
2    0.000000                  0.0               0.0                 0.0   
3    0.000000                  0.0               0.0                 0.0   
4    0.015385                  0.0               0.0                 0.0   

   Auto Workshop  BBQ Joint  Baby Store  Bagel Shop    Bakery  Bank       Bar  \
0       0.000000   0.017241         0.0    0.017241  0.034483   0.0  0.000000   
1       0.000000   0.000000         0.0    0.000000  0.045455   0.0  0.045455   
2       0.052632   0.000000         0.0    0.000000  0.000000   0.0  0.000000   
3       0.000000   0.000000         0.0    0.000000  0.000000   0.0  0.066667   
4       0.000000   0.000000         0.0    0.000000  0.000000   0.0  0.030769   

   Baseball Stadium  Basketball Stadium     Beach  Bed & Breakfast  Beer Bar  \
0               0.0            0.017241  0.017241              0.0  0.034483   
1               0.0            0.000000  0.000000              0.0  0.000000   
2               0.0            0.000000  0.000000              0.0  0.000000   
3               0.0            0.000000  0.000000              0.0  0.000000   
4               0.0            0.000000  0.000000              0.0  0.000000   

   Beer Store  Belgian Restaurant    Bistro  Boat or Ferry  Bookstore  \
0         0.0                 0.0  0.017241       0.000000        0.0   
1         0.0                 0.0  0.000000       0.000000        0.0   
2         0.0                 0.0  0.000000       0.000000        0.0   
3         0.0                 0.0  0.000000       0.066667        0.0   
4         0.0                 0.0  0.000000       0.000000        0.0   

   Boutique  Brazilian Restaurant  Breakfast Spot   Brewery  Bubble Tea Shop  \
0  0.000000                   0.0        0.017241  0.000000         0.000000   
1  0.000000                   0.0        0.090909  0.000000         0.000000   
2  0.000000                   0.0        0.000000  0.052632         0.000000   
3  0.066667                   0.0        0.000000  0.000000         0.000000   
4  0.000000                   0.0        0.000000  0.000000         0.030769   

   Building  Burger Joint  Burrito Place  Bus Line   Butcher      Café  \
0       0.0      0.000000       0.000000       0.0  0.000000  0.034483   
1       0.0      0.000000       0.045455       0.0  0.000000  0.136364   
2       0.0      0.000000       0.052632       0.0  0.052632  0.000000   
3       0.0      0.000000       0.000000       0.0 

In [36]:
toronto_grouped_clustering = toronto_grouped.drop('Neighborhoods', 1)

In [37]:
toronto_grouped_clustering.head()

Afghan Restaurant   Airport  Airport Food Court  Airport Lounge  \
0                0.0  0.000000            0.000000        0.000000   
1                0.0  0.000000            0.000000        0.000000   
2                0.0  0.000000            0.000000        0.000000   
3                0.0  0.066667            0.066667        0.133333   
4                0.0  0.000000            0.000000        0.000000   

   Airport Service  Airport Terminal  American Restaurant  Antique Shop  \
0              0.0          0.000000                  0.0           0.0   
1              0.0          0.000000                  0.0           0.0   
2              0.0          0.000000                  0.0           0.0   
3              0.2          0.066667                  0.0           0.0   
4              0.0          0.000000                  0.0           0.0   

   Aquarium  Art Gallery  Art Museum  Arts & Crafts Store  Asian Restaurant  \
0       0.0     0.017241    0.000000                  0.0               0.0   
1       0.0     0.000000    0.000000                  0.0               0.0   
2       0.0     0.000000    0.000000                  0.0               0.0   
3       0.0     0.000000    0.000000                  0.0               0.0   
4       0.0     0.000000    0.015385                  0.0               0.0   

   Athletics & Sports  Auto Workshop  BBQ Joint  Baby Store  Bagel Shop  \
0                 0.0       0.000000   0.017241         0.0    0.017241   
1                 0.0       0.000000   0.000000         0.0    0.000000   
2                 0.0       0.052632   0.000000         0.0    0.000000   
3                 0.0       0.000000   0.000000         0.0    0.000000   
4                 0.0       0.000000   0.000000         0.0    0.000000   

     Bakery  Bank       Bar  Baseball Stadium  Basketball Stadium     Beach  \
0  0.034483   0.0  0.000000               0.0            0.017241  0.017241   
1  0.045455   0.0  0.045455               0.0            0.000000  0.000000   
2  0.000000   0.0  0.000000               0.0            0.000000  0.000000   
3  0.000000   0.0  0.066667               0.0            0.000000  0.000000   
4  0.000000   0.0  0.030769               0.0            0.000000  0.000000   

   Bed & Breakfast  Beer Bar  Beer Store  Belgian Restaurant    Bistro  \
0              0.0  0.034483         0.0                 0.0  0.017241   
1              0.0  0.000000         0.0                 0.0  0.000000   
2              0.0  0.000000         0.0                 0.0  0.000000   
3              0.0  0.000000         0.0                 0.0  0.000000   
4              0.0  0.000000         0.0                 0.0  0.000000   

   Boat or Ferry  Bookstore  Boutique  Brazilian Restaurant  Breakfast Spot  \
0       0.000000        0.0  0.000000                   0.0        0.017241   
1       0.000000        0.0  0.000000                   0.0        0.090909   
2       0.000000        0.0  0.000000                   0.0        0.000000   
3       0.066667        0.0  0.066667                   0.0        0.000000   
4       0.000000        0.0  0.000000                   0.0        0.000000   

    Brewery  Bubble Tea Shop  Building  Burger Joint  Burrito Place  Bus Line  \
0  0.000000         0.000000       0.0      0.000000       0.000000       0.0   
1  0.000000         0.000000       0.0      0.000000       0.045455       0.0   
2  0.052632         0.000000       0.0      0.000000       0.052632       0.0   
3  0.000000         0.000000       0.0      0.000000       0.000000       0.0   
4  0.000000         0.030769       0.0      0.030769       0.000000       0.0   

    Butcher      Café  Cajun / Creole Restaurant  Camera Store  Candy Store  \
0  0.000000  0.034483                        0.0           0.0          0.0   
1  0.000000  0.136364                        0.0           0.0          0.0   
2  0.052632  0.000000                        0.0           0.0          0.0   
3  0.00

In [38]:
toronto_grouped_clustering.shape

(39, 233)

In [39]:
X = toronto_grouped_clustering.values

Run k-means to cluster the Toronto areas into 5 clusters.

In [40]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3])

In [41]:
neighborhoods_venues_sorted.head()

Neighborhoods 1st Most Common Venue  \
0                                        Berczy Park           Coffee Shop   
1       Brockton, Parkdale Village, Exhibition Place                  Café   
2  Business reply mail Processing Centre, South C...    Light Rail Station   
3  CN Tower, King and Spadina, Railway Lands, Har...       Airport Service   
4                                 Central Bay Street           Coffee Shop   

  2nd Most Common Venue 3rd Most Common Venue  4th Most Common Venue  \
0          Cocktail Bar    Seafood Restaurant                 Bakery   
1           Coffee Shop        Breakfast Spot  Performing Arts Venue   
2            Smoke Shop           Pizza Place             Restaurant   
3        Airport Lounge       Harbor / Marina                    Bar   
4        Sandwich Place    Italian Restaurant    Japanese Restaurant   

  5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
0            Restaurant                  Café           Cheese Shop   
1                Bakery             Nightclub             Pet Store   
2               Butcher               Brewery  Fast Food Restaurant   
3           Coffee Shop   Rental Car Location      Sculpture Garden   
4                  Café       Thai Restaurant      Department Store   

  8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
0              Beer Bar      Department Store              Nightclub  
1          Climbing Gym            Restaurant          Burrito Place  
2         Auto Workshop                   Spa         Farmers Market  
3         Boat or Ferry              Boutique       Airport Terminal  
4                   Bar           Salad Place        Bubble Tea Shop

In [42]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = borough_toronto 

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhoods'), on='Neighborhood')

toronto_merged.head() # check the last columns!

PostalCode          Borough                    Neighborhood   Latitude  \
0        M4E     East Toronto                     The Beaches  43.676357   
1        M4K     East Toronto    The Danforth West, Riverdale  43.679557   
2        M4L     East Toronto  India Bazaar, The Beaches West  43.668999   
3        M4M     East Toronto                 Studio District  43.659526   
4        M4N  Central Toronto                   Lawrence Park  43.728020   

   Longitude  Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
0 -79.293031               0     Health Food Store      Asian Restaurant   
1 -79.352188               3      Greek Restaurant    Italian Restaurant   
2 -79.315572               3                  Park        Sandwich Place   
3 -79.340923               3                  Café           Coffee Shop   
4 -79.388790               0                  Park           Swim School   

  3rd Most Common Venue   4th Most Common Venue 5th Most Common Venue  \
0          Neighborhood                   Trail                   Pub   
1           Coffee Shop  Furniture / Home Store             Bookstore   
2  Fast Food Restaurant             Pizza Place                   Pub   
3               Brewery               Gastropub                Bakery   
4              Bus Line    Fast Food Restaurant    Falafel Restaurant   

  6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
0           Yoga Studio               Dog Run                 Diner   
1            Restaurant        Ice Cream Shop           Yoga Studio   
2         Burrito Place            Restaurant    Italian Restaurant   
3   American Restaurant           Yoga Studio     Convenience Store   
4           Event Space  Ethiopian Restaurant     Electronics Store   

         9th Most Common Venue 10th Most Common Venue  
0               Discount Store    Distribution Center  
1                          Spa    Japanese Restaurant  
2            Fish & Chips Shop           Intersection  
3               Sandwich Place            Cheese Shop  
4  Eastern European Restaurant             Donut Shop

In [43]:
toronto_merged.head()

PostalCode          Borough                    Neighborhood   Latitude  \
0        M4E     East Toronto                     The Beaches  43.676357   
1        M4K     East Toronto    The Danforth West, Riverdale  43.679557   
2        M4L     East Toronto  India Bazaar, The Beaches West  43.668999   
3        M4M     East Toronto                 Studio District  43.659526   
4        M4N  Central Toronto                   Lawrence Park  43.728020   

   Longitude  Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
0 -79.293031               0     Health Food Store      Asian Restaurant   
1 -79.352188               3      Greek Restaurant    Italian Restaurant   
2 -79.315572               3                  Park        Sandwich Place   
3 -79.340923               3                  Café           Coffee Shop   
4 -79.388790               0                  Park           Swim School   

  3rd Most Common Venue   4th Most Common Venue 5th Most Common Venue  \
0          Neighborhood                   Trail                   Pub   
1           Coffee Shop  Furniture / Home Store             Bookstore   
2  Fast Food Restaurant             Pizza Place                   Pub   
3               Brewery               Gastropub                Bakery   
4              Bus Line    Fast Food Restaurant    Falafel Restaurant   

  6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
0           Yoga Studio               Dog Run                 Diner   
1            Restaurant        Ice Cream Shop           Yoga Studio   
2         Burrito Place            Restaurant    Italian Restaurant   
3   American Restaurant           Yoga Studio     Convenience Store   
4           Event Space  Ethiopian Restaurant     Electronics Store   

         9th Most Common Venue 10th Most Common Venue  
0               Discount Store    Distribution Center  
1                          Spa    Japanese Restaurant  
2            Fish & Chips Shop           Intersection  
3               Sandwich Place            Cheese Shop  
4  Eastern European Restaurant             Donut Shop

#### let's visualize the resulting clusters

In [44]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters

#### Cluster 1 (Health Food Store, Swim School , Park) 

In [45]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

Neighborhood  Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
0    The Beaches               0     Health Food Store      Asian Restaurant   
4  Lawrence Park               0                  Park           Swim School   

  3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
0          Neighborhood                 Trail                   Pub   
4              Bus Line  Fast Food Restaurant    Falafel Restaurant   

  6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
0           Yoga Studio               Dog Run                 Diner   
4           Event Space  Ethiopian Restaurant     Electronics Store   

         9th Most Common Venue 10th Most Common Venue  
0               Discount Store    Distribution Center  
4  Eastern European Restaurant             Donut Shop

#### Cluster 2 (Park, Farmers Market)

In [46]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

Neighborhood  Cluster Labels 1st Most Common Venue  \
8  Moore Park, Summerhill East               1                  Park   

  2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
8           Yoga Studio          Dessert Shop        Farmers Market   

  5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
8    Falafel Restaurant           Event Space  Ethiopian Restaurant   

  8th Most Common Venue        9th Most Common Venue 10th Most Common Venue  
8     Electronics Store  Eastern European Restaurant             Donut Shop

#### Cluster 3 (Park, Plyaground, Trail) 

In [47]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

Neighborhood  Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
10     Rosedale               2                  Park            Playground   

   3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
10                 Trail         Deli / Bodega           Event Space   

   6th Most Common Venue 7th Most Common Venue        8th Most Common Venue  \
10  Ethiopian Restaurant     Electronics Store  Eastern European Restaurant   

   9th Most Common Venue 10th Most Common Venue  
10            Donut Shop       Doner Restaurant

#### Cluster 4  (mostly business areas with Restaurants with different cultures, Café,  Supermarkets etc.)

In [48]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

Neighborhood  Cluster Labels  \
1                        The Danforth West, Riverdale               3   
2                      India Bazaar, The Beaches West               3   
3                                     Studio District               3   
5                                    Davisville North               3   
6                   North Toronto West, Lawrence Park               3   
7                                          Davisville               3   
9   Summerhill West, Rathnelly, South Hill, Forest...               3   
11                        St. James Town, Cabbagetown               3   
12                               Church and Wellesley               3   
13                          Regent Park, Harbourfront               3   
14                           Garden District, Ryerson               3   
15                                     St. James Town               3   
16                                        Berczy Park               3   
17                                 Central Bay Street               3   
18                           Richmond, Adelaide, King               3   
19  Harbourfront East, Union Station, Toronto Islands               3   
20           Toronto Dominion Centre, Design Exchange               3   
21                     Commerce Court, Victoria Hotel               3   
23    Forest Hill North & West, Forest Hill Road Park               3   
24                The Annex, North Midtown, Yorkville               3   
25                     University of Toronto, Harbord               3   
26          Kensington Market, Chinatown, Grange Park               3   
27  CN Tower, King and Spadina, Railway Lands, Har...               3   
28                                     Stn A PO Boxes               3   
29             First Canadian Place, Underground city               3   
30                                           Christie               3   
31                       Dufferin, Dovercourt Village               3   
32                           Little Portugal, Trinity               3   
33       Brockton, Parkdale Village, Exhibition Place               3   
34                      High Park, The Junction South               3   
35                             Parkdale, Roncesvalles               3   
36                                 Runnymede, Swansea               3   
37        Queen's Park, Ontario Provincial Government               3   
38  Business reply mail Processing Centre, South C...               3   

   1st Most Common Venue 2nd Most Common Venue  3rd Most Common Venue  \
1       Greek Restaurant    Italian Restaurant            Coffee Shop   
2                   Park        Sandwich Place   Fast Food Restaurant   
3                   Café           Coffee Shop                Brewery   
5                    Gym                 Hotel       Department Store   
6            Coffee Shop        Clothing Store            Yoga Studio   
7            Pizza Place          Dessert Shop         Sandwich Place   
9                    Pub           Coffee Shop            Pizza Place   
11                Bakery           Pizza Place            Coffee Shop   
12           Coffee Shop      Sushi Restaurant    Japanese Restaurant   
13           Coffee Shop                  Park                 Bakery   
14        Clothing Store           Coffee Shop                   Café   
15           Coffee Shop                  Café              Gastropub   
16           Coffee Shop          Cocktail Bar     Seafood Restaurant   
17           Coffee Shop        Sandwich Place     Italian Restaurant   
18           Coffee Shop            Restaurant                   Café   
19           Coffee Shop              Aquarium                  Hotel   
20           Coffee Shop                 Hotel                   Café   
21           Coffee Shop                  Café             Restaurant   
23         Jewelry Store                 Trail     Mexican Restaurant   
24                  Café 

#### Cluster 5 Garden & Pool

In [49]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

Neighborhood  Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
22     Roselawn               4                  Pool          Home Service   

   3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
22                Garden           Yoga Studio      Department Store   

   6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
22           Event Space  Ethiopian Restaurant     Electronics Store   

          9th Most Common Venue 10th Most Common Venue  
22  Eastern European Restaurant             Donut Shop

### Observations:
- Most of the neighborhoods fall into Cluster 4 where the Airport and mostly business areas with Restaurants with different cultures, Café, Supermarkets etc. 
- Cluster 2 where Farmers Market, park, and Yoga Sessions.
- If your concern is healthy food and fresh air that is Cluster 1. 
- Energitic cluster (Cluster 3) that's because Park, Plyaground and Trail.
- Lastly Swimming Pool and Garden in Cluster 5.

![](1.png)

![](2.png)